In [2]:
from pyfuseki import FusekiUpdate
import httpx
from rdflib import Graph
from pysolr import Solr

SOLR

In [2]:
solrAcervo = Solr('http://localhost:8983/solr/authority/', timeout=10)
solrAcervo.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">111</int>\n</lst>\n</response>\n'

In [2]:
solrAcervo = Solr('http://localhost:8983/solr/collection/', timeout=10)
solrAcervo.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">18</int>\n</lst>\n</response>\n'

DELETE GRAPHS

In [4]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authority')

d = """DELETE { graph <https://bibliokeia.com/authorities/PersonalName/bka-2> { ?s ?p ?o } } 
        WHERE {
        graph <https://bibliokeia.com/authorities/PersonalName/bka-2> {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

# solrAcervo = Solr('http://localhost:8983/solr/collection/', timeout=10)
# solrAcervo.delete(q="*:*",  commit=True)

{'statusCode': 200, 'message': 'Update succeeded'}

In [3]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authority')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

solrAcervo = Solr('http://localhost:8983/solr/authority/', timeout=10)
solrAcervo.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">3</int>\n</lst>\n</response>\n'

In [5]:
upThesaurus = FusekiUpdate('http://localhost:3030', 'authority')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upThesaurus.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authority/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">2</int>\n</lst>\n</response>\n'

In [2]:
upThesaurus = FusekiUpdate('http://localhost:3030', 'collection')

d = """DROP DEFAULT }"""

response = upThesaurus.run_sparql("DROP DEFAULT")
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
q = """SELECT ?g  
        {graph ?g {?s ?p ?o}} 
        group by ?g 
        ORDER BY DESC(?g)
        LIMIT 1"""

response = queryAcervo.run_sparql(q)
r = response.convert()
bindings = r['results']['bindings']

WORK BK

In [ ]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/work/bk-1"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('bk-1.nt')

WORK LOC

In [ ]:
g = Graph()
g.parse("http://id.loc.gov/resources/works/1893399.rdf")
g.serialize('LOC.nt')

INSTANCE BK

In [ ]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/instance/bk-37"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('instance.nt')

INSTANCE LOC

In [ ]:
g = Graph()
g.parse("http://id.loc.gov/resources/instances/748300.bibframe.rdf")
g.serialize('instanceLOC.nt')

ITEMS BK

In [ ]:
uri = 'http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/item/bk-38'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('item.nt')

ITEMS LOC

In [ ]:
g = Graph()
g.parse("http://id.loc.gov/resources/items/748300-051-11.bibframe.rdf")
g.serialize('itemLOC.nt')

NAMES BK

In [ ]:
uri = "http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/names/n79138841"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('nameBK.nt')

In [ ]:
def UpdateInstance(instance_id, item_id):
    fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

    up = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        INSERT DATA
            { GRAPH  <https://bibliokeia.com/resources/instance/"""+instance_id+"""> { 
                    <https://bibliokeia.com/resources/instance/"""+instance_id+""">  
                    bf:hasItem
                    <https://bibliokeia.com/resources/item/"""+item_id+"""> } }"""

    response = fuseki_update.run_sparql(up)
    return response



In [ ]:
response = UpdateInstance("bk-", item_id)

SUBJECTS BK

In [ ]:
token = "sh85084414"
uri = f'http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/subjects/{token}'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('subjects.nt')

SERIE LOC

In [ ]:
uri = "http://id.loc.gov/resources/hubs/3035b85d-c7b5-649a-aa12-05ce677f9f11.bibframe.rdf"
g = Graph()
g.parse(uri)
g.serialize('serie.nt')

SERIE BK

In [ ]:

uri = f'http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/hub/bk-5'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('serieBK.nt')

NAMES LOC

In [ ]:
uri = "http://id.loc.gov/authorities/names/n79138841.madsrdf.rdf"
g = Graph()
g.parse(uri)
g.serialize('names.nt')